In [1]:
import numpy as np
import pandas as pd
from micasense import imageset

In [2]:
imgset = imageset.ImageSet.from_directory("/media/davis/UBUNTU 20_0/Easten 050122 M300/0002SET")

Loading ImageSet from: /media/davis/UBUNTU 20_0/Easten 050122 M300/0002SET


In [3]:
data, columns = imgset.as_nested_lists()
df = pd.DataFrame.from_records(data, columns=columns)

In [4]:
def check_calibrated(capture):
    if capture.panels_in_all_expected_images():
        return True
    capture.clear_image_data()
    return False

In [5]:
    for capture in imgset.captures:
        ci = capture.images[0].capture_id
        df.loc[df.capture_id == ci, "capture"] = capture

In [6]:
df["calibrated"] = False
df.calibrated.iloc[:10] = df.capture.iloc[:10].apply(check_calibrated)
df.calibrated.iloc[-10:] = df.capture.iloc[-10:].apply(check_calibrated)

df.calibrated

/home/davis/.local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


0       True
1       True
2       True
3      False
4      False
       ...  
390    False
391    False
392     True
393     True
394     True
Name: calibrated, Length: 395, dtype: bool

In [7]:
df

,timestamp,latitude,longitude,altitude,capture_id,dls-yaw,dls-pitch,dls-roll,irr-475,irr-560,irr-668,irr-840,irr-717,capture,calibrated
0,2022-05-01 18:50:00.214243+00:00,40.651590,-104.993526,1553.285,wmW6Ovyglr7f3fte4v8j,-2.861374,-0.227556,-0.008809,0.671917,0.575687,0.624080,0.378787,0.566405,<micasense.capture.Capture object at 0x7fa5305...,True
1,2022-05-01 18:50:03.973934+00:00,40.651591,-104.993527,1553.319,pEAA0e9bAccdQCPNoQqo,-2.879719,-0.231877,-0.000182,0.677606,0.579780,0.627843,0.381185,0.571736,<micasense.capture.Capture object at 0x7fa5091...,True
2,2022-05-01 18:50:07.764530+00:00,40.651592,-104.993529,1553.381,dqlcOLQfKHJqdGoncpe0,-2.906203,-0.240109,0.003390,0.678770,0.580520,0.629626,0.381993,0.571969,<micasense.capture.Capture object at 0x7fa5091...,True
3,2022-05-01 18:51:09.543810+00:00,40.651599,-104.993523,1553.680,aZc4htnDo4FDpZfbF8yi,-3.005834,-0.174524,-0.026523,0.797196,0.676911,0.726001,0.427084,0.655633,<micasense.capture.Capture object at 0x7fa5091...,False
4,2022-05-01 18:52:03.725354+00:00,40.651614,-104.993429,1592.947,Q4b7qQ5K7Y9T8vH52rZI,2.831001,-0.096044,0.161941,0.799710,0.676803,0.719237,0.429604,0.671434,<micasense.capture.Capture object at 0x7fa5091...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,2022-05-01 19:07:17.845346+00:00,40.651575,-104.993519,1559.618,aQDuu2ABmXjHmMqYGFXe,2.271071,0.405651,0.053434,1.281296,1.072486,1.162344,0.699422,1.053836,<micasense.capture.Capture object at 0x7fa5091...,False
391,2022-05-01 19:07:23.684240+00:00,40.651583,-104.993499,1559.289,nrGKOQTfgiwihDs6mljL,0.259467,0.489496,0.030988,0.800456,0.692916,0.755643,0.471064,0.695798,<micasense.capture.Capture object at 0x7fa5091...,False
392,2022-05-01 19:08:15.672521+00:00,40.651597,-104.993539,1556.564,ZVbsTAz79aiQE0CmD49b,-2.464294,-0.072894,0.149037,1.105981,0.949384,1.024134,0.615710,0.947390,<micasense.capture.Capture object at 0x7fa5091...,True
393,2022-05-01 19:08:19.132080+00:00,40.651597,-104.993538,1556.345,y2I7l1YvUP7O6J0OhKnh,-2.497420,-0.095350,0.154298,1.104590,0.947363,1.022000,0.613737,0.942102,<micasense.capture.Capture object at 0x7fa5091...,True


In [8]:
cutoff_altitude = df.altitude.mean()-3.0*df.altitude.std()
df["valid_altitude"] = df["altitude"] > cutoff_altitude
cutoff_altitude

1566.714817144407

In [9]:
df["timestamp_diff"] = df.timestamp.diff()
df[df.timestamp_diff == df.timestamp_diff.max()]

,timestamp,latitude,longitude,altitude,capture_id,dls-yaw,dls-pitch,dls-roll,irr-475,irr-560,irr-668,irr-840,irr-717,capture,calibrated,valid_altitude,timestamp_diff
3,2022-05-01 18:51:09.543810+00:00,40.651599,-104.993523,1553.68,aZc4htnDo4FDpZfbF8yi,-3.005834,-0.174524,-0.026523,0.797196,0.676911,0.726001,0.427084,0.655633,<micasense.capture.Capture object at 0x7fa5091...,False,False,0 days 00:01:01.779280


In [12]:
gr = df.groupby((df.valid_altitude!=df.valid_altitude.shift()).cumsum())
gr.groups

{1: [0, 1, 2, 3], 2: [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, ...], 3: [380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394]}

In [11]:
df.timestamp.diff().std()

Timedelta('0 days 00:00:05.150550510')